In [5]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.metrics import confusion_matrix
from IPython.display import display
import warnings

# Suppress all warnings (including FutureWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore")

# ==============================================================================
# 1. GENERATE PREDICTIONS AND CREATE RESULTS DATAFRAME
# ==============================================================================

# Use the same data loading and feature separation as in model_training.ipynb
from pathlib import Path
import os

# Get project root assuming this notebook is in src/notebooks/
notebook_dir = Path(os.getcwd())
root_dir = notebook_dir.parent.parent

train_path = root_dir / 'artifacts' / 'train.parquet'
test_path  = root_dir / 'artifacts' / 'test.parquet'

for p in [train_path, test_path]:
    if not p.exists():
        raise FileNotFoundError(f"File not found: {p.resolve()}.\nPlease check that the file exists and the path is correct.")

df_train = pd.read_parquet(train_path)
df_test  = pd.read_parquet(test_path)

X_test  = df_test.drop(columns=['target'])
y_test  = df_test['target']

# Load the best model pipeline from disk instead of retraining
import joblib
model_path = root_dir / "reports" / "modeling" / "best_model" / "model.pkl"
if not model_path.exists():
    raise FileNotFoundError(f"Trained model not found at {model_path}. Please ensure the model is saved at this path.")
final_pipeline = joblib.load(model_path)

try:
    y_pred_test_proba = final_pipeline.predict_proba(X_test)[:, 1]
    y_pred_test = (y_pred_test_proba > 0.5).astype(int)
except Exception as e:
    raise RuntimeError(f"Error during prediction: {e}")

# Create a consolidated DataFrame for analysis
results_df = X_test.copy()
results_df['actual'] = y_test.values
results_df['predicted'] = y_pred_test

print("--- Dataframe for Manual Error Analysis ---")
display(results_df.head())

# ==============================================================================
# 2. CALCULATE GLOBAL FPR AND FNR
# ==============================================================================
print("\n--- Global Error Rates ---")
cm = confusion_matrix(results_df['actual'], results_df['predicted'])
if cm.shape == (2, 2):
    tn, fp, fn, tp = cm.ravel()
else:
    # Handle edge case where only one class is present in y_test
    tn = fp = fn = tp = 0
    if cm.shape == (1, 1):
        if results_df['actual'].iloc[0] == 0:
            tn = cm[0, 0]
        else:
            tp = cm[0, 0]
    elif cm.shape == (2, 1):
        tn = cm[0, 0]
        fn = cm[1, 0]
    elif cm.shape == (1, 2):
        tn = cm[0, 0]
        fp = cm[0, 1]

fpr = fp / (fp + tn) if (fp + tn) > 0 else np.nan
fnr = fn / (fn + tp) if (fn + tp) > 0 else np.nan
print(f"Global False Positive Rate (FPR): {fpr:.4f}")
print(f"Global False Negative Rate (FNR): {fnr:.4f}")

# ==============================================================================
# 3. MANUAL ANALYSIS PER FEATURE
# ==============================================================================
print("\n--- False Positive and False Negative Rates per Feature ---")

# Identify features (exclude 'actual' and 'predicted')
feature_cols = [col for col in results_df.columns if col not in ['actual', 'predicted']]
numerical_features = results_df[feature_cols].select_dtypes(include=np.number).columns.tolist()
categorical_features = results_df[feature_cols].select_dtypes(exclude=np.number).columns.tolist()

# --- Analyze Categorical Features ---
for feature in categorical_features:
    print(f"\nAnalysis for: {feature}")
    try:
        cm_by_category = results_df.groupby(feature).apply(
            lambda x: pd.Series({
                'tn': ((x['actual'] == 0) & (x['predicted'] == 0)).sum(),
                'fp': ((x['actual'] == 0) & (x['predicted'] == 1)).sum(),
                'fn': ((x['actual'] == 1) & (x['predicted'] == 0)).sum(),
                'tp': ((x['actual'] == 1) & (x['predicted'] == 1)).sum()
            })
        )
        # Avoid division by zero
        cm_by_category['FPR'] = cm_by_category.apply(lambda r: r['fp'] / (r['fp'] + r['tn']) if (r['fp'] + r['tn']) > 0 else np.nan, axis=1)
        cm_by_category['FNR'] = cm_by_category.apply(lambda r: r['fn'] / (r['fn'] + r['tp']) if (r['fn'] + r['tp']) > 0 else np.nan, axis=1)
        display(cm_by_category[['FPR', 'FNR']].round(4))
    except Exception as e:
        print(f"  > Skipping {feature}: {e}")

# --- Analyze Numerical Features (by binning) ---
for feature in numerical_features:
    print(f"\nAnalysis for: {feature}")
    try:
        # Only bin if there are at least 2 unique values
        if results_df[feature].nunique() < 2:
            print(f"  > Skipping {feature}: Not enough unique values for binning.")
            continue
        bins = pd.qcut(results_df[feature], q=5, duplicates='drop', retbins=True)[1]
        if len(bins) < 2:
            print(f"  > Skipping {feature}: Not enough unique values for binning.")
            continue
        results_df[f'{feature}_bin'] = pd.cut(results_df[feature], bins=bins, include_lowest=True)
        cm_by_bin = results_df.groupby(f'{feature}_bin').apply(
            lambda x: pd.Series({
                'tn': ((x['actual'] == 0) & (x['predicted'] == 0)).sum(),
                'fp': ((x['actual'] == 0) & (x['predicted'] == 1)).sum(),
                'fn': ((x['actual'] == 1) & (x['predicted'] == 0)).sum(),
                'tp': ((x['actual'] == 1) & (x['predicted'] == 1)).sum()
            })
        )
        cm_by_bin['FPR'] = cm_by_bin.apply(lambda r: r['fp'] / (r['fp'] + r['tn']) if (r['fp'] + r['tn']) > 0 else np.nan, axis=1)
        cm_by_bin['FNR'] = cm_by_bin.apply(lambda r: r['fn'] / (r['fn'] + r['tp']) if (r['fn'] + r['tp']) > 0 else np.nan, axis=1)
        display(cm_by_bin[['FPR', 'FNR']].round(4))
    except Exception as e:
        print(f"  > Skipping {feature}: {e}")

--- Dataframe for Manual Error Analysis ---


,Age_Years,Systolic_BP,Diastolic_BP,BMI,Pulse_Pressure,Demographics_Score,Clinical_Biomarkers_Score,Lifestyle_Score,Glucose_Level,Cholesterol_Level,BP_level,Sex,Smoking_Status,Alcohol_Intake,Physical_Activity,actual,predicted
0,0.6,1.00,0.0,0.784260,2.0,0.524864,2.742571,-0.195794,Above Normal,Well Above Normal,3,2,0,0,1,1,1
1,0.7,0.50,1.0,0.441637,0.0,0.440739,0.866537,0.000000,Normal,Normal,3,1,0,0,0,1,1
2,-0.4,-0.50,-2.0,-0.201103,1.0,-0.197329,0.101145,-0.195794,Above Normal,Normal,1,2,0,0,1,0,0
3,0.4,-0.50,-1.0,0.201376,0.0,0.236655,-0.265735,0.000000,Normal,Normal,1,1,0,0,0,0,0
4,0.4,-0.25,-1.0,0.588138,0.5,0.324516,-0.026941,-0.195794,Normal,Normal,1,1,0,0,1,0,0



--- Global Error Rates ---
Global False Positive Rate (FPR): 0.2293
Global False Negative Rate (FNR): 0.3010

--- False Positive and False Negative Rates per Feature ---

Analysis for: Glucose_Level


,FPR,FNR
Glucose_Level,,
Normal,0.2008,0.3249
Above Normal,0.3538,0.2470
Well Above Normal,0.5412,0.1349



Analysis for: Cholesterol_Level


...

,FPR,FNR
Smoking_Status_bin,,
"(-0.001, 1.0]",0.2293,0.301



Analysis for: Alcohol_Intake


,FPR,FNR
Alcohol_Intake_bin,,
"(-0.001, 1.0]",0.2293,0.301



Analysis for: Physical_Activity


,FPR,FNR
Physical_Activity_bin,,
"(-0.001, 1.0]",0.2293,0.301


In [1]:
# ==============================================================================
# 1. IMPORTS
# ==============================================================================
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from lightgbm import LGBMClassifier
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE

# Install responsibleai and raiwidgets if not already installed
%pip install responsibleai raiwidgets

# Core libraries for the Responsible AI Dashboard
from responsibleai import RAIInsights
from raiwidgets import RAIInsights as RAIWidgets

# Suppress warnings for cleaner output
import warnings
warnings.filterwarnings('ignore')

# ==============================================================================
# 2. DATA PREPARATION
# ==============================================================================
# Assumes 'df' is a pandas DataFrame loaded in your environment.
if 'df' not in locals() or df.empty:
    raise NameError("DataFrame 'df' is not defined or is empty. Please load your data before running this code.")

# Separate features (X) and target (y)
X = df.drop(columns=['target'])
y = df['target']

# --- Identify features for the dashboard ---
# The RAI dashboard needs to know which features are categorical.
numerical_features = X.select_dtypes(include=np.number).columns.tolist()
categorical_features = X.select_dtypes(exclude=np.number).columns.tolist()

# --- Data Split (as per your original pipeline) ---
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.10, random_state=42, stratify=y)
X_train, X_valid, y_train, y_valid = train_test_split(X_temp, y_temp, test_size=0.1111, random_state=42, stratify=y_temp)

# ==============================================================================
# 3. TRAIN THE FINAL MODEL
# ==============================================================================
# This replicates the final model training step from your previous code.
print("--- Training the final LGBMClassifier model... ---")

# Preprocessor to transform the data
preprocessor = ColumnTransformer([
    ('num', StandardScaler(), numerical_features),
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
])

# Final model pipeline with SMOTE
final_pipeline = ImbPipeline([
    ('preprocessor', preprocessor),
    ('balancer', SMOTE(random_state=42)),
    ('classifier', LGBMClassifier(random_state=42, n_jobs=-1, verbose=-1)) # Using default params for brevity
])

final_pipeline.fit(X_temp, y_temp)

print("✅ Model training complete.")

# ==============================================================================
# 4. GENERATE THE RESPONSIBLE AI DASHBOARD
# ==============================================================================
print("\n--- Generating the Responsible AI Dashboard... ---")

try:
    # 1. Instantiate the RAIInsights object
    insights = RAIInsights(
        model=final_pipeline,
        train=X_temp,
        test=X_test,
        target_column='target',
        task_type='classification',
        categorical_features=categorical_features
    )

    # 2. Add the desired analysis components
    insights.add_error_analysis()
    insights.add_feature_importance()
    insights.add_counterfactuals(
        desired_class='opposite', # The target is the opposite class
        total_cf_samples=10,      # Number of counterfactuals to generate per sample
        features_to_vary='all'
    )
    # Causal analysis can be added here if needed, but it's computationally intensive.
    # insights.add_causal(...)

    # 3. Compute the insights
    insights.compute()

    # 4. Launch the interactive dashboard
    dashboard = RAIWidgets(insights)
    print("✅ Dashboard generated. You can now explore the insights.")

except Exception as e:
    print(f"❌ An error occurred during dashboard generation: {e}")
    print("Ensure all required dependencies (responsibleai, raiwidgets) are installed.")

^C
Note: you may need to restart the kernel to use updated packages.


ModuleNotFoundError: No module named 'responsibleai'

  Using cached responsibleai-0.36.0-py3-none-any.whl.metadata (18 kB)
  Using cached dice_ml-0.11-py3-none-any.whl.metadata (20 kB)
  Using cached econml-0.16.0-cp311-cp311-win_amd64.whl.metadata (37 kB)
  Using cached statsmodels-0.13.5-cp311-cp311-win_amd64.whl.metadata (9.5 kB)
  Using cached erroranalysis-0.5.5-py3-none-any.whl.metadata (2.1 kB)
  Using cached interpret_community-0.32.0-py3-none-any.whl.metadata (4.4 kB)
  Using cached numpy-1.26.2-cp311-cp311-win_amd64.whl.metadata (61 kB)
  Using cached numba-0.58.1-cp311-cp311-win_amd64.whl.metadata (2.8 kB)
  Using cached pandas-1.5.3-cp311-cp311-win_amd64.whl.metadata (12 kB)
  Using cached scikit_learn-1.5.1-cp311-cp311-win_amd64.whl.metadata (12 kB)
  Using cached semver-2.13.0-py2.py3-none-any.whl.metadata (5.0 kB)
  Using cached ml_wrappers-0.6.0-py3-none-any.whl.metadata (1.9 kB)
  Using cached networkx-2.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached raiutils-0.4.2-py3-none-any.whl.metadata (1.4 kB)
  Using cached l